In [50]:
import pandas as pd


In [51]:
#Defining paths as strings to change whenever it's necessary instead of hard coding paths in rest of code
main_directory = "../input_files/conjunto_de_datos/"
catalog_directory = "../input_files/catalogos/"
file_dictionary="../input_files/diccionario_de_datos/diccionario_datos_tviaje.csv"

In [64]:
#Reading from fil
viaje_file = "tviaje.csv"
output_file = "../output_files/movilidad.csv"
viaje_df = pd.read_csv(main_directory+viaje_file)
Dictionary=pd.read_csv(file_dictionary)

Deleting all rows not having an specific source or destination

In [65]:
#Getting all rows 
not_sour_or_dest = viaje_df.query('p5_7_7==99 or p5_12_7==99')
#pd.concat (viaje_df, not_sour_or_dest).drop_duplicates(keep=False)




# Survey and response type

In [4]:
Path_Catalogs=[]   #Save only path with the target data
Descrip_Columns=[] #To only have columns with descip different and skip descrip with: "veces", "personas", "paradas" y "Años cumplidos"

for column in range(0,82):  #Summary of question and kand of anwser
    valor=Dictionary['nombre_campo'][column]
    try:
        temporalfile=Dictionary['catálogo'][column]+str('.csv')
        temporalpath=f'{catalog_directory}{temporalfile}'
    
        temporal_variable=pd.read_csv(temporalpath)
        
    
        print(f'''columna: {int(column)}:{valor}
        {temporal_variable.head(5)}''')
        
        if len(temporal_variable)>2:
            if column+1!=67:  # Is the only case diferent of the other condition
                if temporal_variable['descrip'][1]!=temporal_variable['descrip'][2]:
                    Path_Catalogs.append(temporalpath)
                    Descrip_Columns.append(column)
        else:
            Path_Catalogs.append(temporalpath)
            Descrip_Columns.append(column) 
        
    except:
        print(f'''columna: {int(column)}:{valor}
        sin catalogo''')
       
    print(f'''------------------------------------------------------------------
    ''')
       

columna: 0:Identificador de viaje
        sin catalogo
------------------------------------------------------------------
    
columna: 1:Identificador de persona
        sin catalogo
------------------------------------------------------------------
    
columna: 2:Pregunta 5.3 Clasificación del día de viaje
           cve                                   descrip
0    1  Viaje realizado entre semana (Sección V)
1    2    Viaje realizado en sábado (Sección VI)
------------------------------------------------------------------
    
columna: 3:Pregunta 5.8 número de viaje
        sin catalogo
------------------------------------------------------------------
    
columna: 4:Pregunta 5.6 ¿Qué tipo de lugar es el origen de su viaje?
           cve                                       descrip
0    1                                      Su hogar
1    2                                       Escuela
2    3                                       Oficina
3    4                              Fábr

columna: 50:Pregunta 5.15 Desde que inició hasta que llegó a su destino final ¿cuántas veces usó  17 Mototaxi?
           cve descrip
0    1     Vez
1    2   Veces
2    3   Veces
3    4   Veces
4    5   Veces
------------------------------------------------------------------
    
columna: 51:Pregunta 5.14 Dígame por favor sí utilizó 18 Transporte  escolar como medio de transporte
           cve     descrip
0    1     Utilizó
1    2  No utilizó
------------------------------------------------------------------
    
columna: 52:Pregunta 5.15 Desde que inició hasta que llegó a su destino final ¿cuántas veces usó 18 Transporte  escolar?
           cve descrip
0    1     Vez
1    2   Veces
2    3   Veces
3    4   Veces
4    5   Veces
------------------------------------------------------------------
    
columna: 53:Pregunta 5.14 Dígame por favor sí utilizó 19 Transporte  de personal como medio de transporte
           cve     descrip
0    1     Utilizó
1    2  No utilizó
------------------

# Merging all tables 

## Getting the general route information

In [5]:
#Read the trip classification
p5_3_cat =pd.read_csv(catalog_directory+"p5_3.csv")

#Merge1
viaje1 =pd.merge(viaje_df, p5_3_cat, how='left', left_on='p5_3', right_on='cve')

viaje1=viaje1.rename(columns={"descrip": "clasif_viaje"})
viaje1.shape

(531594, 84)

In [6]:
#Read the route's starting point
p5_6_cat =pd.read_csv(catalog_directory+"p5_6.csv")

#Merge 2
viaje2 =pd.merge(viaje1, p5_6_cat, how='left', left_on='p5_6', right_on='cve')

viaje2=viaje2.rename(columns={"descrip": "lugar_origen"})
viaje2.shape

(531594, 86)

In [7]:
#Read the catalogue: source municipality
p5_7_6_cat =pd.read_csv(catalog_directory+"p5_7_6.csv")

#Merge 3
viaje3 =pd.merge(viaje2, p5_7_6_cat, how='left', left_on=['p5_7_7', 'p5_7_6'], right_on=['ent','mun'])

viaje3=viaje3.rename(columns={"descrip": "municipio_origen"})
viaje3.shape

(531594, 89)

In [8]:
#Read the catalogue source entity
p5_7_7_cat =pd.read_csv(catalog_directory+"p5_7_7.csv")

#Merge 4
viaje4 =pd.merge(viaje3, p5_7_7_cat, how='left', left_on='p5_7_7', right_on='cve')

viaje4=viaje4.rename(columns={"descrip": "estado_origen"})
viaje4.shape


(531594, 91)

In [9]:
#Read the catalogue source zone 
dato_origen_cat =pd.read_csv(catalog_directory+"dto_origen.csv")

#Merge 5
viaje5 =pd.merge(viaje4, dato_origen_cat, how='left', left_on='dto_origen', right_on='cve')
viaje5=viaje5.rename(columns={"descrip": "colonia_origen"})

#Deleting duplicate records
viaje5.drop_duplicates(keep='first', inplace=True)

viaje5.shape
#viaje5.query('id_via == 398883')


(531594, 93)

In [10]:
#Read the catalogue destination type
p5_11a_cat =pd.read_csv(catalog_directory+"p5_11a.csv")

#Merge 6
viaje6 =pd.merge(viaje5, p5_11a_cat, how='left', left_on='p5_11a', right_on='cve')
viaje6=viaje6.rename(columns={"descrip": "tipo_lugar_destino_"})
viaje6.shape

#viaje6.head()

(531594, 95)

In [11]:
#Read the catalogue municipality destination
p5_12_6_cat =pd.read_csv(catalog_directory+"p5_12_6.csv")

#Merge 7
viaje7 =pd.merge(viaje6, p5_12_6_cat, how='left', left_on=['p5_12_7','p5_12_6'], right_on=['ent','mun'])
viaje7=viaje7.rename(columns={"descrip": "municipio_destino"})
viaje7.shape

(531594, 98)

In [12]:
#Read the catalogue destination state
p5_12_7_cat =pd.read_csv(catalog_directory+"p5_12_7.csv")

#Merge 8
viaje8 =pd.merge(viaje7, p5_12_7_cat, how='left', left_on='p5_12_7', right_on='cve')

viaje8=viaje8.rename(columns={"descrip": "estado_destino"})
viaje8.shape

(531594, 100)

In [13]:
#Read the catalogue destination zone
dto_dest_cat =pd.read_csv(catalog_directory+"dto_dest.csv")

#Merge 9
viaje9 =pd.merge(viaje8, dto_dest_cat, how='left', left_on='dto_dest', right_on='cve')

#Removing duplicated row
viaje9.drop_duplicates(keep='first', inplace=True)
viaje9=viaje9.rename(columns={"descrip": "colonia_destino"})
viaje9.shape

(531594, 102)

In [14]:
#Read the catalogue trip purpose
p5_13_cat =pd.read_csv(catalog_directory+"p5_13.csv")

#Merge 10
viaje10 =pd.merge(viaje9, p5_13_cat, how='left', left_on='p5_13', right_on='cve')
viaje10=viaje10.rename(columns={"descrip": "proposito_viaje"})
viaje10.shape

(531594, 104)

In [15]:
#Renaming dataframe columns
viaje10=viaje10.rename(columns={"p5_9_1": "hora_comienzo", "p5_9_2": "minuto_comienzo","p5_10_1":"hora_termino","p5_10_2":"minuto_termino"})

In [16]:
#Reducing dataframe to have only trip source-destination info as columns
viaje10=viaje10[["id_via","id_soc","clasif_viaje","lugar_origen","municipio_origen","estado_origen","colonia_origen","tipo_lugar_destino_","municipio_destino","estado_destino","colonia_destino","proposito_viaje","hora_comienzo","minuto_comienzo","hora_termino","minuto_termino"]]


In [17]:
viaje10_list = viaje10.columns

In [18]:
viaje10.shape

(531594, 16)

In [19]:
#Getting catalog files to get values of the following columns 'p5_14_04', 'p5_15_04',
       #'p5_14_05', 'p5_15_05', 'p5_14_06', 'p5_15_06', 'p5_14_07', 'p5_15_07',
       #'p5_14_08', 'p5_15_08', 'p5_14_09', 'p5_15_09', 'p5_14_10', 'p5_15_10',
       #'p5_14_11', 'p5_15_11', 'p5_14_12', 'p5_15_12', 'p5_14_13', 'p5_15_13',
       #'p5_14_14', 'p5_15_14'

#Creating an iterator to get form catalog name and retrieve it as DataFrame 
#As catalogs have consecutive numbers in their names

#file prefix
file_pref = 'p5'

#List to store all catalogs' names to delete them after merging
column_names = []
#Storing columns from merged actions to be deleted
column_merged_names = []

#DataFrame where all catalogs will be merged
viajes_merged_df = viaje_df.copy()

#Iterating to form catalog_name
for x in range(1,15):
    for y in range(14,16):
        #If x < 10, adding a '0' previous to x value
        if x<10:
            catalog_name = f"{file_pref}_{y}_0{x}"
        else:
            catalog_name = f"{file_pref}_{y}_{x}"

        #Catalog file to create DataFrame
        catalog_file = f"{catalog_name}.csv"
        
        #Suffix to add to every new merged column from catalog
        suffix='_'+catalog_name
        
        #Adding catalog name to list of catalog names to be deleted after merging.
        column_names.append(catalog_name)
        
        #If y=14, 'cve' column needs to be deleted after merging catalog DataFrames
        #Else (y=15), 'descrip' column needs to be deleted after merging catalog DataFrames
        if y==14:
            column_merged_names.append('cve'+suffix)
        else:
            column_merged_names.append('descrip'+suffix)
        
        #Creating DataFrame using each catalog file
        catalog_df = pd.read_csv(catalog_directory+catalog_file)
        
                
        #Starting merging each DataFrame. PLease note that catolog file's name (catalog_name) is the same as column name in viaje_df DataFrame
        #All catalogs have the key on 'cve' field
        
        viajes_merged_df = pd.merge(viajes_merged_df, catalog_df, how='left', left_on=catalog_name, right_on='cve', suffixes=('', suffix))
        



## Considera las columnas del DataFrame orginal de la x a la 81

In [20]:
myList_LV=[46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,65,75,76,77,78,79,80,81]

In [21]:
myList2_LV=[]
for index in range(len(Descrip_Columns)):
    for x in range(len(myList_LV)):
        if myList_LV[x]==Descrip_Columns[index]:
            myList2_LV.append(index)


In [22]:
viajes_merged_df2 = viaje_df.copy() # For use as alternative

for column in range(len(myList2_LV)): #sub-rutine to merge and drop only the columns in my List_LV
    
   #Set up auxiliar to use merge
    column_left=viajes_merged_df2.columns[Descrip_Columns[myList2_LV[column]]] 
    
    catalog_df2=pd.read_csv(Path_Catalogs[myList2_LV[column]])
    
    column_right=catalog_df2.columns[0]
    
    
    #Merge
    viajes_merged_df2=pd.merge(
                               viajes_merged_df2, catalog_df2,
                               how='left', left_on=column_left,
                               right_on=column_right                       
                              )
    
    #Sep ut to use drop
    columns_to_drop=[column_left]
    if len(catalog_df2.columns)==2:
        columns_to_drop.append(f'cve')
    else:
        for c in range(len(catalog_df2.columns)-1):
            columns_to_drop.append(f'{catalog_df2.columns[c]}')
    
    
    #Drop
    viajes_merged_df2.drop(columns=columns_to_drop,inplace=True)
    
    #Rename the column with the value of catalogo and rename
    viajes_merged_df2=viajes_merged_df2.rename(columns={'descrip':column_left})
         
    listC=viajes_merged_df2.columns
    
    #Sep up to have the same orden in columns as the original Data
    LisN=[]

    for i in range(len(listC)):
        if i==Descrip_Columns[myList2_LV[column]]:
            LisN.append(listC[-1])
        elif i>Descrip_Columns[myList2_LV[column]]:
            LisN.append(listC[i-1])
        else:
            LisN.append(listC[i])
    
    
    viajes_merged_df2=viajes_merged_df2[LisN]
    
    


In [23]:
Columns_Merge_LV=[]
for column in myList_LV:
    Columns_Merge_LV.append(viajes_merged_df2.columns[column])

In [24]:
viajes_merged_df2=viajes_merged_df2[Columns_Merge_LV]

In [25]:
viajes_merged_df2

,p5_15_15,p5_14_16,p5_15_16,p5_14_17,p5_15_17,p5_14_18,p5_15_18,p5_14_19,p5_15_19,p5_14_20,...,p5_21_1,p5_21_2,p5_25,estrato,factor,upm_dis,est_dis,tloc,sexo,edad
0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
1,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
2,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
3,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
4,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531589,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,26
531590,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,51
531591,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,51
531592,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,...,NaN,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Mujer,49


# Starts Cleaning Process

## Drop Repeated Columns

In [26]:
#Using catalog names list to drop columns with keys to catalogs (the original columns named as catalog names) and new columns
#named like 'cvep5...'

#for column_name in column_names:
for (column_name, column_merged_name) in zip(column_names, column_merged_names):    
    #The 'cve' column of first catalog merged, which name is added without suffix, it keeps only 'cve' as column names, so
    #we need to validate it, to drop column differently
    #print(column_name)
    if column_name=='p5_14_01':
        viajes_merged_df.drop(columns=[column_name, 'cve'],inplace=True)
    else:
    #Dropping column having the catalog name as name, and the new merged catalog column having 'cve_' as preffix
        viajes_merged_df.drop(columns=[column_name,column_merged_name],inplace=True)


In [27]:
Columns_to_Drop3=[]
for newColumn in range(len(Columns_Merge_LV)):
    Columns_to_Drop3.append(f'{Columns_Merge_LV[newColumn]}_drop')



In [28]:
viajes_merged_df = pd.merge(viajes_merged_df, viajes_merged_df2,
                             how='outer', on=viajes_merged_df.index,
                             left_index=True,
                             suffixes=('_drop', ''))

In [29]:
viajes_merged_df.set_index('key_0', drop=True, append=False, inplace=True, verify_integrity=False)

In [30]:
viajes_merged_df=viajes_merged_df.drop(columns=Columns_to_Drop3)

In [31]:
viajes_merged_df.shape

(531594, 82)

In [32]:
viaje10.columns

Index(['id_via', 'id_soc', 'clasif_viaje', 'lugar_origen', 'municipio_origen',
       'estado_origen', 'colonia_origen', 'tipo_lugar_destino_',
       'municipio_destino', 'estado_destino', 'colonia_destino',
       'proposito_viaje', 'hora_comienzo', 'minuto_comienzo', 'hora_termino',
       'minuto_termino'],
      dtype='object')

In [33]:
viajes_merged_df3 = pd.merge(viaje10, viajes_merged_df, how='left', left_index=True, right_index=True, suffixes=('','_drop'))
viajes_merged_df3

,id_via,id_soc,clasif_viaje,lugar_origen,municipio_origen,estado_origen,colonia_origen,tipo_lugar_destino_,municipio_destino,estado_destino,...,p5_21_1,p5_21_2,p5_25,estrato,factor,upm_dis,est_dis,tloc,sexo,edad
0,2936,1268,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
1,2937,1268,Viaje realizado entre semana (Sección V),Oficina,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Su hogar,Cuauhtémoc,Ciudad de México,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
2,2938,1268,Viaje realizado en sábado (Sección VI),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Otra vivienda,Cuauhtémoc,Ciudad de México,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
3,2939,1268,Viaje realizado en sábado (Sección VI),Otra vivienda,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Su hogar,Cuauhtémoc,Ciudad de México,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
4,2940,1269,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,...,NaN,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531589,530057,199596,Viaje realizado entre semana (Sección V),Oficina,Tlalpan,Ciudad de México,Villa Olimpica,Su hogar,Tizayuca,Hidalgo,...,NaN,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,26
531590,530058,199597,Viaje realizado en sábado (Sección VI),Su hogar,Tizayuca,Hidalgo,Tizayuca,Fábrica o taller,Tecámac,México,...,NaN,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,51
531591,530059,199597,Viaje realizado en sábado (Sección VI),Fábrica o taller,Tecámac,México,Base Áerea Sta Lucia-Cuautlalpan,Su hogar,Tizayuca,Hidalgo,...,NaN,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,51
531592,530060,199598,Viaje realizado en sábado (Sección VI),Su hogar,Tizayuca,Hidalgo,Tizayuca,"Comercio, mercado, tienda o centro comercial",Tizayuca,Hidalgo,...,NaN,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Mujer,49


In [34]:
viajes_merged_df3.columns


Index(['id_via', 'id_soc', 'clasif_viaje', 'lugar_origen', 'municipio_origen',
       'estado_origen', 'colonia_origen', 'tipo_lugar_destino_',
       'municipio_destino', 'estado_destino', 'colonia_destino',
       'proposito_viaje', 'hora_comienzo', 'minuto_comienzo', 'hora_termino',
       'minuto_termino', 'id_via_drop', 'id_soc_drop', 'p5_3', 'n_via', 'p5_6',
       'p5_7_6', 'p5_7_7', 'dto_origen', 'p5_9_1', 'p5_9_2', 'p5_10_1',
       'p5_10_2', 'p5_11a', 'p5_12_6', 'p5_12_7', 'dto_dest', 'p5_13',
       'p5_14_15', 'p5_22', 'p5_23', 'p5_24', 'p5_26', 'p5_27_1', 'p5_27_2',
       'p5_27_3', 'p5_27_4', 'p5_27_5', 'p5_27_6', 'p5_27_7', 'p5_27_8',
       'descrip', 'cve_p5_15_01', 'descrip_p5_14_02', 'cve_p5_15_02',
       'descrip_p5_14_03', 'cve_p5_15_03', 'descrip_p5_14_04', 'cve_p5_15_04',
       'descrip_p5_14_05', 'cve_p5_15_05', 'descrip_p5_14_06', 'cve_p5_15_06',
       'descrip_p5_14_07', 'cve_p5_15_07', 'descrip_p5_14_08', 'cve_p5_15_08',
       'descrip_p5_14_09', 'cve_p

In [35]:
#Dropping columns having the same values as viaje10
drop_cols_viaje10 = ['p5_3', 'n_via', 'p5_6', 'p5_7_6','p5_7_7', 'dto_origen', 
                     'p5_9_1', 'p5_9_2', 'p5_10_1', 'p5_10_2',
                     'p5_11a', 'p5_12_6', 'p5_12_7', 'dto_dest','p5_13', 'id_via_drop', 'id_soc_drop']

viajes_merged_df3.drop(columns=drop_cols_viaje10, inplace=True)
viajes_merged_df3.columns

Index(['id_via', 'id_soc', 'clasif_viaje', 'lugar_origen', 'municipio_origen',
       'estado_origen', 'colonia_origen', 'tipo_lugar_destino_',
       'municipio_destino', 'estado_destino', 'colonia_destino',
       'proposito_viaje', 'hora_comienzo', 'minuto_comienzo', 'hora_termino',
       'minuto_termino', 'p5_14_15', 'p5_22', 'p5_23', 'p5_24', 'p5_26',
       'p5_27_1', 'p5_27_2', 'p5_27_3', 'p5_27_4', 'p5_27_5', 'p5_27_6',
       'p5_27_7', 'p5_27_8', 'descrip', 'cve_p5_15_01', 'descrip_p5_14_02',
       'cve_p5_15_02', 'descrip_p5_14_03', 'cve_p5_15_03', 'descrip_p5_14_04',
       'cve_p5_15_04', 'descrip_p5_14_05', 'cve_p5_15_05', 'descrip_p5_14_06',
       'cve_p5_15_06', 'descrip_p5_14_07', 'cve_p5_15_07', 'descrip_p5_14_08',
       'cve_p5_15_08', 'descrip_p5_14_09', 'cve_p5_15_09', 'descrip_p5_14_10',
       'cve_p5_15_10', 'descrip_p5_14_11', 'cve_p5_15_11', 'descrip_p5_14_12',
       'cve_p5_15_12', 'descrip_p5_14_13', 'cve_p5_15_13', 'descrip_p5_14_14',
       'cve_p5_15

In [36]:
viajes_merged_df3['p5_14_15'].head(5)

0    2
1    2
2    2
3    2
4    2
Name: p5_14_15, dtype: int64

In [37]:
viajes_merged_df3.shape

(531594, 81)

In [38]:
viajes_merged_df3.columns

Index(['id_via', 'id_soc', 'clasif_viaje', 'lugar_origen', 'municipio_origen',
       'estado_origen', 'colonia_origen', 'tipo_lugar_destino_',
       'municipio_destino', 'estado_destino', 'colonia_destino',
       'proposito_viaje', 'hora_comienzo', 'minuto_comienzo', 'hora_termino',
       'minuto_termino', 'p5_14_15', 'p5_22', 'p5_23', 'p5_24', 'p5_26',
       'p5_27_1', 'p5_27_2', 'p5_27_3', 'p5_27_4', 'p5_27_5', 'p5_27_6',
       'p5_27_7', 'p5_27_8', 'descrip', 'cve_p5_15_01', 'descrip_p5_14_02',
       'cve_p5_15_02', 'descrip_p5_14_03', 'cve_p5_15_03', 'descrip_p5_14_04',
       'cve_p5_15_04', 'descrip_p5_14_05', 'cve_p5_15_05', 'descrip_p5_14_06',
       'cve_p5_15_06', 'descrip_p5_14_07', 'cve_p5_15_07', 'descrip_p5_14_08',
       'cve_p5_15_08', 'descrip_p5_14_09', 'cve_p5_15_09', 'descrip_p5_14_10',
       'cve_p5_15_10', 'descrip_p5_14_11', 'cve_p5_15_11', 'descrip_p5_14_12',
       'cve_p5_15_12', 'descrip_p5_14_13', 'cve_p5_15_13', 'descrip_p5_14_14',
       'cve_p5_15

In [39]:
#Delete columns having values not needed for our analysis
viajes_merged_df3.drop(columns=['p5_22', 'p5_23', 'p5_24', 'p5_26',
       'p5_27_1', 'p5_27_2', 'p5_27_3', 'p5_27_4', 'p5_27_5', 'p5_27_6',
       'p5_27_7', 'p5_27_8', 'p5_19', 'p5_20', 'p5_21_1', 'p5_21_2',
       'p5_25'], inplace=True)

## Adding remaining columns to a list

In [40]:
#Listing all remaining columns of DataFrame
mylist = list(viajes_merged_df3)
mylist

['id_via',
 'id_soc',
 'clasif_viaje',
 'lugar_origen',
 'municipio_origen',
 'estado_origen',
 'colonia_origen',
 'tipo_lugar_destino_',
 'municipio_destino',
 'estado_destino',
 'colonia_destino',
 'proposito_viaje',
 'hora_comienzo',
 'minuto_comienzo',
 'hora_termino',
 'minuto_termino',
 'p5_14_15',
 'descrip',
 'cve_p5_15_01',
 'descrip_p5_14_02',
 'cve_p5_15_02',
 'descrip_p5_14_03',
 'cve_p5_15_03',
 'descrip_p5_14_04',
 'cve_p5_15_04',
 'descrip_p5_14_05',
 'cve_p5_15_05',
 'descrip_p5_14_06',
 'cve_p5_15_06',
 'descrip_p5_14_07',
 'cve_p5_15_07',
 'descrip_p5_14_08',
 'cve_p5_15_08',
 'descrip_p5_14_09',
 'cve_p5_15_09',
 'descrip_p5_14_10',
 'cve_p5_15_10',
 'descrip_p5_14_11',
 'cve_p5_15_11',
 'descrip_p5_14_12',
 'cve_p5_15_12',
 'descrip_p5_14_13',
 'cve_p5_15_13',
 'descrip_p5_14_14',
 'cve_p5_15_14',
 'p5_15_15',
 'p5_14_16',
 'p5_15_16',
 'p5_14_17',
 'p5_15_17',
 'p5_14_18',
 'p5_15_18',
 'p5_14_19',
 'p5_15_19',
 'p5_14_20',
 'p5_15_20',
 'p5_18',
 'estrato',
 'fact

## Validate all columns have at least one valid value (not null)

In [41]:
#Validating all columns have at least one not null value through column iteration
for column in mylist:
    if viajes_merged_df3[column].isnull().values.all()==True:
        viajes_merged_df3.drop(columns=column, inplace=True)
        


## Replacing all NaN values with zero

In [42]:
#Replacing all NaN values with zero
viajes_merged_df3.fillna(0)

,id_via,id_soc,clasif_viaje,lugar_origen,municipio_origen,estado_origen,colonia_origen,tipo_lugar_destino_,municipio_destino,estado_destino,...,p5_14_20,p5_15_20,p5_18,estrato,factor,upm_dis,est_dis,tloc,sexo,edad
0,2936,1268,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,...,No utilizó,0.0,0,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
1,2937,1268,Viaje realizado entre semana (Sección V),Oficina,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Su hogar,Cuauhtémoc,Ciudad de México,...,No utilizó,0.0,0,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
2,2938,1268,Viaje realizado en sábado (Sección VI),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Otra vivienda,Cuauhtémoc,Ciudad de México,...,No utilizó,0.0,0,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
3,2939,1268,Viaje realizado en sábado (Sección VI),Otra vivienda,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Su hogar,Cuauhtémoc,Ciudad de México,...,No utilizó,0.0,0,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
4,2940,1269,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,...,No utilizó,0.0,0,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531589,530057,199596,Viaje realizado entre semana (Sección V),Oficina,Tlalpan,Ciudad de México,Villa Olimpica,Su hogar,Tizayuca,Hidalgo,...,No utilizó,0.0,0,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,26
531590,530058,199597,Viaje realizado en sábado (Sección VI),Su hogar,Tizayuca,Hidalgo,Tizayuca,Fábrica o taller,Tecámac,México,...,No utilizó,0.0,0,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,51
531591,530059,199597,Viaje realizado en sábado (Sección VI),Fábrica o taller,Tecámac,México,Base Áerea Sta Lucia-Cuautlalpan,Su hogar,Tizayuca,Hidalgo,...,No utilizó,0.0,0,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,51
531592,530060,199598,Viaje realizado en sábado (Sección VI),Su hogar,Tizayuca,Hidalgo,Tizayuca,"Comercio, mercado, tienda o centro comercial",Tizayuca,Hidalgo,...,No utilizó,0.0,0,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Mujer,49


## Renaming column Titles

In [43]:
#Renaming columns
viajes_merged_df3.rename(columns={
'descrip' : '¿Auto?',
'cve_p5_15_01' : '# Auto',
'descrip_p5_14_02' : '¿Colectivo/Micro?',
'cve_p5_15_02' : '# Colectivo/Micro',
'descrip_p5_14_03' : '¿Taxi (App)?',
'cve_p5_15_03' : '# Taxi (App)',
'descrip_p5_14_04':'¿Taxi?',
'cve_p5_15_04':'#Taxi',
'descrip_p5_14_05':'¿Metro?',
'cve_p5_15_05':'# Metro',
'descrip_p5_14_06':'¿Autobus RTP?',
'cve_p5_15_06':'# Autobus RTP',
'descrip_p5_14_07':'¿Bicicleta?',
'cve_p5_15_07':'#Veces Bicicleta',
'descrip_p5_14_08':'¿Autobus?',
'cve_p5_15_08':'# Autobus',
'descrip_p5_14_09':'¿Moto?',
'cve_p5_15_09':'# Moto',
'descrip_p5_14_10':'¿Trolebus?',
'cve_p5_15_10':'# Trolebus',
'descrip_p5_14_11':'¿Metrobus?',
'cve_p5_15_11':'# Metrobus',
'descrip_p5_14_12':'¿Tren ligero?',
'cve_p5_15_12':'# Tren ligero',
'descrip_p5_14_13':'¿Tren suburbano?',
'cve_p5_15_13':'# Tren suburbano',
'descrip_p5_14_14':'¿Camino?',
'cve_p5_15_14':'# Camino',
'p5_14_15': '¿Mexicable?',
'p5_15_15': '#Mexicable',
'p5_14_16':'¿Bicitaxi?',
'p5_15_16': '# Bicitaxi',
'p5_14_17': '¿Mototaxi?',
'p5_15_17':  '#Mototaxi',
'p5_14_18':  '¿Transporte escolar?',
'p5_15_18': '#Transporte escolar',
'p5_14_19': '¿Transporte de personal?',
'p5_15_19': '#Transporte de personal',
'p5_14_20': '¿Otro medio?',
'p5_15_20': '# Otro medio',
'p5_18': '¿Condujo?',
}, inplace=True)

In [44]:
viajes_merged_df3.columns

Index(['id_via', 'id_soc', 'clasif_viaje', 'lugar_origen', 'municipio_origen',
       'estado_origen', 'colonia_origen', 'tipo_lugar_destino_',
       'municipio_destino', 'estado_destino', 'colonia_destino',
       'proposito_viaje', 'hora_comienzo', 'minuto_comienzo', 'hora_termino',
       'minuto_termino', '¿Mexicable?', '¿Auto?', '# Auto',
       '¿Colectivo/Micro?', '# Colectivo/Micro', '¿Taxi (App)?',
       '# Taxi (App)', '¿Taxi?', '#Taxi', '¿Metro?', '# Metro',
       '¿Autobus RTP?', '# Autobus RTP', '¿Bicicleta?', '#Veces Bicicleta',
       '¿Autobus?', '# Autobus', '¿Moto?', '# Moto', '¿Trolebus?',
       '# Trolebus', '¿Metrobus?', '# Metrobus', '¿Tren ligero?',
       '# Tren ligero', '¿Tren suburbano?', '# Tren suburbano', '¿Camino?',
       '# Camino', '#Mexicable', '¿Bicitaxi?', '# Bicitaxi', '¿Mototaxi?',
       '#Mototaxi', '¿Transporte escolar?', '#Transporte escolar',
       '¿Transporte de personal?', '#Transporte de personal', '¿Otro medio?',
       '# Otro medi

In [45]:
#viajes_merged_df3.rename(columns={'p5_18': '¿Condujo?'}, inplace=True)

In [46]:
#viajes_merged_df3.columns

In [47]:
#viajes_merged_df3.shape

In [48]:
#Exporting clean DataFrame to a csv file
#viajes_merged_df3.to_csv(output_file)

## Saving clean DataFrame in memory to be used by routines allocated in other Jupyter notebooks

In [49]:
%store viajes_merged_df3
del viajes_merged_df3

Stored 'viajes_merged_df3' (DataFrame)
